# Final Project Report for CS 175, Spring 2020
**Project Title:** Water Filled Lung Detector

**Project Number:** Group 14

**Student Name(s)**

Allan Tran, 61735904. allannt@uci.edu

Jason Davis, 22336416, jasonbd@uci.edu

Eva Dai, 94015611, ydai8@uci.edu

First, load any source code we need and import them.

In [3]:
%%bash

# Using PyTorch helper functions to simplify training
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

Branch 'v0.5.1' set up to track remote branch 'v0.5.1' from 'origin'.


Cloning into 'vision'...
Switched to a new branch 'v0.5.1'


In [5]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 90kB/s 


### **Warning: The most recent version of pycocotools from https://github.com/cocodataset/cocoapi.git is needed to import the modules in the first import cell and to run the Faster RCNN part of this notebook**

The next code block attempts to update the pycocotools library on your computer if you already have it installed.

In [0]:
%%bash

git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi
cp PythonAPI/pycocotools C:/Python36/Lib/site-packages/ -r

In [0]:
from engine import train_one_epoch, evaluate

In [0]:
import torch
import utils
from torch import optim
import torchvision
from src.dataset import PneumoniaDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from src.classifier import Classifier

import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.nn.functional import relu as Relu
from torch import sigmoid

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit

import pandas as pd

## Classifier:

Here we start to load the classifier model and train it with a small sample of our data for demostration purpose.

In [0]:
root = "./data"

In [3]:
model = Classifier();
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Classifier(
  (convInput): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32to64): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv64): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv64to128): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv128): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linearto128): Linear(in_features=2097152, out_features=128, bias=True)
  (linearto1): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [0]:
dataset = PneumoniaDataset(root, True)
validation_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:20])
validation_dataset = torch.utils.data.Subset(validation_dataset, indices[20:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True, num_workers=1,
        collate_fn=utils.collate_fn)

validation_data_loader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)


test_dataset = PneumoniaDataset(root, True, 'test')

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
gpu_dtype = torch.cuda.FloatTensor
loss_fn = nn.BCEWithLogitsLoss().type(gpu_dtype)

In [0]:
def train(train_data, dtype, model, loss_fn, optimizer, num_epochs=1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, (image, target) in enumerate(train_data):
            imgs = torch.tensor([img.numpy() for img in image], dtype=torch.float32)
            x_var = Variable(imgs.type(dtype))
            l = []
            for tar in target:
              if len(tar['labels']) > 1:
                # temp = [list(tar['labels'])[0]]
                l.append([1])
              else:
                if tar['labels'] == 2:
                  l.append([1])
                else:
                  l.append([0])
            y_var = torch.tensor(l).cuda() # Variable(labels.type(dtype).cuda())
            scores = model(imgs.cuda())
            loss = loss_fn(scores, y_var.float())
            if (t + 1) % 5 == 0:
              print('t = %d, loss = %.4f' % (t + 1, loss.item()))
              print(y_var)
              print(scores)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [6]:
train(data_loader, gpu_dtype, model, loss_fn, optimizer, 1)

Starting epoch 1 / 1
t = 5, loss = 0.0000
tensor([[0]], device='cuda:0')
tensor([[-61.6630]], device='cuda:0', grad_fn=<AddmmBackward>)
t = 10, loss = 0.0000
tensor([[0]], device='cuda:0')
tensor([[-11.3769]], device='cuda:0', grad_fn=<AddmmBackward>)
t = 15, loss = 1.5481
tensor([[0]], device='cuda:0')
tensor([[1.3090]], device='cuda:0', grad_fn=<AddmmBackward>)
t = 20, loss = 0.5898
tensor([[0]], device='cuda:0')
tensor([[-0.2187]], device='cuda:0', grad_fn=<AddmmBackward>)


After the training, we can check the accuracy on the validation set.

In [0]:
def check_accuracy(model, loader):
    """if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')"""  
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, (image, target) in enumerate(loader):
        with torch.no_grad():
          imgs = torch.tensor([img.numpy() for img in image], dtype=torch.float32)
          l = []
          metadata = []
          for tar in target:
            if len(tar['labels']) > 1:
              l.append([1])
            else:
              if tar['labels'] == 2:
                l.append([1])
              else:
                l.append([0])
            metadata.append([tar['position']])
          l = torch.tensor(l)
        raw_scores = sigmoid(model(imgs.cuda()))
        scores = torch.tensor([int(raw_scores > 0.37)])
        num_correct += (scores == l).sum()
        num_samples += scores.size(0)
        if (t + 1) % 1 == 0:
            print('t = %d, num_correct = %d, num_samples = %d' % (t + 1, num_correct, num_samples))
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [8]:
check_accuracy(model, validation_data_loader)

t = 1, num_correct = 0, num_samples = 1
t = 2, num_correct = 0, num_samples = 2
t = 3, num_correct = 0, num_samples = 3
t = 4, num_correct = 0, num_samples = 4
t = 5, num_correct = 1, num_samples = 5
t = 6, num_correct = 2, num_samples = 6
Got 2 / 6 correct (33.33)


Now we can save the predictions into a csv file. But for demostration, here we'll print out the predictions instead.

In [0]:
def save_preds(model, loader, dataset):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    pIds = []
    predictions = []
    for index, (image, target) in enumerate(loader):
        with torch.no_grad():
          imgs = torch.tensor([img.numpy() for img in image], dtype=torch.float32)
        scores =  torch.tensor([int(sigmoid(model(imgs.cuda())) > 0.37)])
        patientId = dataset.imgs[index][:-4]
        p = scores.numpy()[0]
        pIds.append(patientId)
        predictions.append(p)

    d = {'patientId': pIds, 'pred': predictions}
    #df = pd.DataFrame(data=d)
    #path = 'drive/My Drive/cs-175-project/predictions/classifier_w_metadata_prediction.csv'
    #df.to_csv(path, index=False)
    print(d)
    

In [10]:
save_preds(model, test_data_loader, test_dataset)

{'patientId': ['0000a175-0e68-4ca4-b1af-167204a7e0bc', '0005d3cc-3c3f-40b9-93c3-46231c3eb813', '000686d7-f4fc-448d-97a0-44fa9c5d3aa6', '000e3a7d-c0ca-4349-bb26-5af2d8993c3d', '00100a24-854d-423d-a092-edcf6179e061', '0015597f-2d69-4bc7-b642-5b5e01534676', '001b0c51-c7b3-45c1-9c17-fa7594cab96e', '0022bb50-bf6c-4185-843e-403a9cc1ea80', '00271e8e-aea8-4f0a-8a34-3025831f1079', '0028450f-5b8e-4695-9416-8340b6f686b0'], 'pred': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


As shown in the validation and test predictions above, we can see the accuracy is very low. It is caused by the small number of the training data we have in this submission. During the actual project, we ran on the entire dataset of 26k images, which would result in a much higher accuracy of 90.1%.

## Faster RCNN:

In [11]:
faster_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = faster_model.roi_heads.box_predictor.cls_score.in_features
faster_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=3)
faster_model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [12]:
train_one_epoch(faster_model, optimizer, data_loader, device, 1, 5)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [1]  [ 0/20]  eta: 0:00:06  lr: 0.001000  loss: 3.8248 (3.8248)  loss_classifier: 1.3701 (1.3701)  loss_box_reg: 0.0010 (0.0010)  loss_objectness: 1.7421 (1.7421)  loss_rpn_box_reg: 0.7116 (0.7116)  time: 0.3299  data: 0.1138  max mem: 6475
Epoch: [1]  [ 5/20]  eta: 0:00:03  lr: 0.001000  loss: 3.7450 (3.0420)  loss_classifier: 1.3701 (1.3862)  loss_box_reg: 0.0004 (0.0070)  loss_objectness: 1.6429 (1.1531)  loss_rpn_box_reg: 0.7116 (0.4957)  time: 0.2167  data: 0.0235  max mem: 6475
Epoch: [1]  [10/20]  eta: 0:00:02  lr: 0.001000  loss: 3.7659 (3.2419)  loss_classifier: 1.3747 (1.3778)  loss_box_reg: 0.0010 (0.0111)  loss_objectness: 1.6821 (1.3136)  loss_rpn_box_reg: 0.7254 (0.5393)  time: 0.2063  data: 0.0153  max mem: 6475
Epoch: [1]  [15/20]  eta: 0:00:01  lr: 0.001000  loss: 3.7659 (3.3336)  loss_classifier: 1.3747 (1.3791)  loss_box_reg: 0.0010 (0.0123)  loss_objectness: 1.6776 (1.3848)  loss_rpn_box_reg: 0.7254 (0.5573)  time: 0.2023  data: 0.0121  max mem: 6475
Epoch: [

This code uses a PyTorch helper function to train the Faster RCNN Model on the dataset.

In [13]:
evaluate(faster_model, validation_data_loader, device)

creating index...
index created!


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Test:  [0/6]  eta: 0:00:01  model_time: 0.0681 (0.0681)  evaluator_time: 0.0033 (0.0033)  time: 0.2196  data: 0.1449  max mem: 6475
Test:  [5/6]  eta: 0:00:00  model_time: 0.0606 (0.0617)  evaluator_time: 0.0033 (0.0053)  time: 0.0974  data: 0.0287  max mem: 6475
Test: Total time: 0:00:00 (0.1087 s / it)
Averaged stats: model_time: 0.0606 (0.0617)  evaluator_time: 0.0033 (0.0053)
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  

We also used a PyTorch helper function to evaluate the model on the validation dataset. Because of the small datasize and using only 1 epoch, the top line showing the mean average precision of our predicted boxes with confidence between 50% and 95% is pretty low, "Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000".

In [0]:
faster_preds = pd.DataFrame(columns = ['patientId', 'PredictionString'])
confidence = 0.5

In [0]:
def get_faster_preds(model, data, testset, device, conf, dataframe):
  model.eval()
  predictions = []
  for i, (x, y) in enumerate(data):
    with torch.no_grad():
      images = list(image.to(device) for image in x)
    preds = model(images)[0]
    pred_str = []
    for index, score in enumerate(preds['scores'].tolist()):
      if score >= conf:
        boxes = preds['boxes'][index].tolist()
        if preds['labels'][index] == 2:
          pred_str.extend([score, boxes[0], boxes[1], boxes[2] - boxes[0], boxes[3] - boxes[1]])
    pred_str = [str(x) for x in pred_str]
    pred_str = ' '.join(pred_str)
    dataframe = dataframe.append({'patientId': testset.imgs[i][:-4], 'PredictionString': pred_str}, ignore_index=True)
  return dataframe

In [16]:
final_preds = get_faster_preds(faster_model, test_data_loader, test_dataset, device, confidence, faster_preds)
print(final_preds)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


                              patientId                                   PredictionString
0  0000a175-0e68-4ca4-b1af-167204a7e0bc  0.5763124227523804 0.0 0.772656261920929 328.2...
1  0005d3cc-3c3f-40b9-93c3-46231c3eb813  0.6492416858673096 0.0 15.478055953979492 247....
2  000686d7-f4fc-448d-97a0-44fa9c5d3aa6  0.632843554019928 876.880859375 99.17414855957...
3  000e3a7d-c0ca-4349-bb26-5af2d8993c3d  0.5587719678878784 0.0 0.0 196.31295776367188 ...
4  00100a24-854d-423d-a092-edcf6179e061  0.6293582916259766 772.3051147460938 0.0 251.6...
5  0015597f-2d69-4bc7-b642-5b5e01534676  0.6138786673545837 785.5523071289062 159.37661...
6  001b0c51-c7b3-45c1-9c17-fa7594cab96e  0.585543155670166 349.5295715332031 0.0 345.61...
7  0022bb50-bf6c-4185-843e-403a9cc1ea80  0.6658321619033813 84.30679321289062 65.343292...
8  00271e8e-aea8-4f0a-8a34-3025831f1079  0.6860902309417725 918.9534301757812 99.247772...
9  0028450f-5b8e-4695-9416-8340b6f686b0  0.6256241798400879 809.3922729492188 52.985641...

Here we can see the output of the model in the of the Kaggle Competition result format. One patient id associated with a string of the confidence of the model for a bounding box, the x and y coordinate of the box, and its width and height